# Tensorflow

## TensorFlow API
* Low Level API와 High Level API(tf.estimator)가 존재
* Low Level API reference - https://www.tensorflow.org/programmers_guide/low_level_intro

## Tensor
* 다차원 array

In [1]:
import numpy as np
t0 = np.array(3)
print(len(t0.shape), t0.shape) # rank 0, shape []
t1 = np.array([1., 2., 3.])
print(len(t1.shape), t1.shape)  # rank 1, shape [3]
t2 = np.array([[1., 2., 3.], [4., 5., 6.]])
print(len(t2.shape), t2.shape) # rank 2, shape [2, 3]
t3 = np.array([[[1., 2., 3.]], [[7., 8., 9.]]])
print(len(t3.shape), t3.shape) # rank 3, shape [2, 1, 3]

0 ()
1 (3,)
2 (2, 3)
3 (2, 1, 3)


## Tensorflow 사용

In [2]:
import tensorflow as tf

In [3]:
sess = tf.Session()
hello = tf.constant('Hello, TensorFlow!')
sess.run(hello)

b'Hello, TensorFlow!'

## Computational Graph
* 노드는 Tensor(들)을 받아 다른 tensor를 내보냄.
* constant : 입력은 없고 단일 output을 내보냄.

In [4]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) #  암시적으로 tf.float32 type
print(node1, node2)

Tensor("Const_1:0", shape=(), dtype=float32) Tensor("Const_2:0", shape=(), dtype=float32)


In [5]:
#total = tf.add(node1, node2)
total = node1 + node2
total

<tf.Tensor 'add:0' shape=() dtype=float32>

* node1, node2는 3,4를 내보내는 노드.
* Session을 통해 그래프를 실행.

In [6]:
sess.run([node1, node2, total])

[3.0, 4.0, 7.0]

## Placeholder
* 외부의 입력을 받음.

In [7]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

In [8]:
adder_node = a + b

In [9]:
# sess.run([adder_node]) # adder_node에 값을 넣어줘야 지 오류가 안남.
x = np.array([3, 10, 15])
y = np.array([4.5, 20, 100])

print(x.shape)
print(y.shape)

feed_dict = {a: x, b: y} # a,b 데이터의 shape가 똑같아야 함.
sess.run([adder_node], feed_dict)

(3,)
(3,)


[array([  7.5,  30. , 115. ], dtype=float32)]

* 머신러닝에서는 학습값을 계속 업데이트 하기 때문에 variable(변수)을 사용.

In [10]:
# Linear regression
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)

linear_model = W * x + b

* variable(변수)은 명시적으로 아래처럼 initialize 해야함.

In [11]:
init = tf.global_variables_initializer()
sess.run(init)

In [12]:
feed_dict = {x: [1, 10, 50, 100]}
sess.run(linear_model, feed_dict)

array([ 0.      ,  2.7     , 14.700001, 29.700003], dtype=float32)

In [13]:
y = tf.placeholder(tf.float32)
squared_delta = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_delta)

tf.summary.scalar('loss', loss)
loss_summary = tf.summary.merge_all()

In [14]:
feed_dict = {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}
sess.run(loss, feed_dict)

23.66

* Grandient descent 를 사용하여 variable을 optimize하는 함수를 제공.

In [15]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_op = optimizer.minimize(loss)

In [16]:
optimizer

In [17]:
train_op

<tf.Operation 'GradientDescent' type=NoOp>

In [18]:
import os
model_save_dir = '%s/model' % os.getcwd()
summary_writer = tf.summary.FileWriter(model_save_dir, graph=sess.graph, flush_secs=3)

In [19]:
print(loss_summary)
for i in range(1000):
    _loss, _loss_summary, _W, _b, _train_op = sess.run([loss, loss_summary, W, b, train_op], feed_dict)
    summary_writer.add_summary(_loss_summary, i)

Tensor("Merge/MergeSummary:0", shape=(), dtype=string)


In [20]:
sess.run([loss, W, b, train_op], feed_dict)

[5.6999738e-11,
 array([-0.99999696], dtype=float32),
 array([0.99999094], dtype=float32),
 None]

# TensorBoard
* 실행방법
<pre>
$ cd model
model$ tensorboard --logdir=.
</pre>
* html5lib upgrade (만약 실행 안되면)
<pre>
sindonglyeol-ui-MacBook-Pro:model ryanshin$ pip install --upgrade html5lib
</pre>

* 모델별로 디렉터리를 나누어 저장하면 (예:gd, adam) tensorboard에 2개 모델에 대한 그래프를 시각적으로 비교할 수 있음.